In this notebook I process the .csv transaction data dump from commsec to do some cash flow analysis and stuff for my mum.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from yahoo_finance import Share

I went to Trading>Confirmations and downloaded a .csv file.  lalala

In [ ]:
confirmations = pd.read_csv('ConfirmationDetails.csv').iloc[:, :11]
confirmations.head(2)

In [ ]:
current_prices = {}

for ticker in confirmations.Security.unique():
    current_prices[ticker] = Share(ticker+'.AX').get_price()

current_prices = pd.DataFrame.from_dict(current_prices, orient='index')
current_prices.columns = ['price']
current_prices.sort_values('price', ascending=False, inplace=True)

I went to Portfolio>Accounts>Transactions and downloaded a .csv of all the transactions there.

Had to make sure that my date range started at June 1, to get all the info.

In [ ]:
transactions = pd.read_csv("Transactions.csv").iloc[:, :7]
transactions.fillna("", inplace=True)
transactions['Balance ($)'] = transactions['Balance ($)']/1000
transactions.rename(columns={'Balance ($)': "Balance ($000)"}, inplace=True)
transactions.head(2)

In [ ]:
divs = pd.read_csv('CSVData.csv', header=None)
divs['Security'] = divs.iloc[:,2].apply(lambda x: x.split()[3])
divs.rename(columns={1: 'Proceeds'}, inplace=True)
divs.head(2)

In [ ]:
big_dict = {}

for ticker in confirmations.Security.unique():
    revenue = confirmations['Net Proceeds ($)'][
        confirmations['Security'] == ticker][
        confirmations['Buy/ Sell']=='S'].sum()

    cost = confirmations['Net Proceeds ($)'][
            confirmations['Security'] == ticker][
            confirmations['Buy/ Sell']=='B'].sum()
    
    tran_num = confirmations[confirmations.Security == ticker].shape[0]
    
    num_bought = confirmations['Units'][(confirmations['Security'] == ticker) & (confirmations['Buy/ Sell'] == 'B')].sum()
    num_sold = confirmations['Units'][(confirmations['Security'] == ticker) & (confirmations['Buy/ Sell'] == 'S')].sum()
    num_holding = num_bought-num_sold
    
    curr_price = float(current_prices.ix[ticker])
    asset_value = curr_price * num_holding
    if divs.Proceeds[divs.Security == ticker].shape[0] == 0:
        dividend = 0
    else: dividend = round(divs.Proceeds[divs.Security == ticker].values.tolist()[0], 2)
            
    prof=round(revenue+int(dividend)-cost, 2)
    net_position = round(prof + asset_value, 2) # is this right? am i double counting something?
    
    big_dict[ticker] = [round(revenue, 2), round(cost, 2), tran_num, num_holding,
                        asset_value, dividend, prof, net_position, curr_price]

In [ ]:
all_info = pd.DataFrame.from_dict(big_dict, orient='index')
all_info.columns = ['Revenue', 'Cost', 'Num Transactions', 'Num Holding', 'Asset Value', 'Dividends', 'Profit', 'Net Position', 'Current Price']
all_info.sort_values('Net Position', ascending=True)

In [ ]:
all_info['Net Position'].sum()

In [ ]:
#     print("""For {ticker}: 
#     ${spent} in total has been spent, ${gained} has been gained from sales and ${divi} from dividends.  
#     {tran_num} transactions were undertaken for {ticker2}.
#     We currently hold {num_holding} units, valued at ${asset_value} total.
#     This represents ${prof} net profit.
#     Your net position on this stock is ${net}.
#     """.format(ticker=ticker,
#                spent=round(cost, 2),
#                gained=round(revenue, 2),
#                divi=int(dividend),
#                prof=prof,
#                tran_num=tran_num,
#                ticker2=ticker,
#                num_holding=num_holding,
#                asset_value=asset_value,
#                net=net_position))